<a href="https://colab.research.google.com/github/kevinbeslia/TB-kelompok-8/blob/main/Prak2_RuleBasedClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#mengimport library dan dataset
import pandas as pd
df = pd.read_csv('persona.csv')
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [3]:
#observasi frekuensi nilai yang unik pada variabel kategori
df["SOURCE"].nunique() 
df["SOURCE"].value_counts()
df["COUNTRY"].value_counts() 

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [4]:
#memeriksa nilai pada variabel yang sama
df.groupby("COUNTRY")["PRICE"].agg({"mean"}) #rincian pendapatan rata-rata negara 
df.groupby(["COUNTRY", 'SOURCE'])["PRICE"].mean() #negara dan sumber rincian rata-rata pendapatan

COUNTRY  SOURCE 
bra      android    34.387029
         ios        34.222222
can      android    33.330709
         ios        33.951456
deu      android    33.869888
         ios        34.268817
fra      android    34.312500
         ios        32.776224
tur      android    36.229437
         ios        33.272727
usa      android    33.760357
         ios        34.371703
Name: PRICE, dtype: float64

In [5]:
#pendapatan rata-rata berdasarkan variabel
agg_df = df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"])["PRICE"].mean().sort_values(ascending=False)
agg_df.head()

COUNTRY  SOURCE   SEX     AGE
bra      android  male    46     59.0
usa      android  male    36     59.0
fra      android  female  24     59.0
usa      ios      male    32     54.0
deu      android  female  36     49.0
Name: PRICE, dtype: float64

In [9]:
#mengubah nama indeks menjadi variabel
agg_df = agg_df.reset_index() #jika salah,operasi tidak dilakukan
agg_df.head() #nomor indeks baru

,index,COUNTRY,SOURCE,SEX,AGE,PRICE
0,0,bra,android,male,46,59.0
1,1,usa,android,male,36,59.0
2,2,fra,android,female,24,59.0
3,3,usa,ios,male,32,54.0
4,4,deu,android,female,36,49.0


In [18]:
#mengubah label 'Age' menjadi varibel kategori
newlabels = ['0_18', '19_23', '24_30', '31_40', '41_70']
agg_df["AGE_CUT"] = pd.cut(x=agg_df["AGE"], bins=[0, 18, 23, 30, 40, 70], labels=newlabels)
agg_df.tail(10)

,index,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CUT
338,338,bra,android,male,23,21.5,19_23
339,339,tur,android,male,21,19.0,19_23
340,340,tur,ios,male,47,19.0,41_70
341,341,bra,ios,female,34,19.0,31_40
342,342,bra,ios,male,47,19.0,41_70
343,343,usa,ios,female,38,19.0,31_40
344,344,usa,ios,female,30,19.0,24_30
345,345,can,android,female,27,19.0,24_30
346,346,fra,android,male,18,19.0,0_18
347,347,deu,android,male,26,9.0,24_30


In [19]:
#mengidentifikasi pelanggan dengan level baru
agg_df["customers_level_based"] = [f"{i[0]}_{i[1]}_{i[2]}_{i[-1]}" for i in agg_df.values]

In [20]:
#memilih baris dan kolom
agg_df = agg_df.loc[:, ["customers_level_based", "PRICE"]].groupby("customers_level_based").agg({"PRICE": "mean"}).sort_values(by="PRICE", ascending=False).reset_index()
agg_df["customers_level_based"].head()

0    0_bra_android_41_70
1    2_fra_android_24_30
2    1_usa_android_31_40
3        3_usa_ios_31_40
4        8_fra_ios_19_23
Name: customers_level_based, dtype: object

In [21]:
#melakukan segmentasi pelanggan baru
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head()

,customers_level_based,PRICE,SEGMENT
0,0_bra_android_41_70,59.0,A
1,2_fra_android_24_30,59.0,A
2,1_usa_android_31_40,59.0,A
3,3_usa_ios_31_40,54.0,A
4,8_fra_ios_19_23,49.0,A


In [22]:
#menjelaskan segmen baru
agg_df.groupby(["SEGMENT"]).agg({"PRICE": ["mean", "max", "sum"]})
agg_df[agg_df["SEGMENT"] == "C"].describe()

,PRICE
count,95.000000
mean,32.933339
std,0.877933
min,31.173913
25%,32.333333
50%,32.913043
75%,33.861004
max,34.000000


In [23]:
#input data untuk menentukan segmen
new_user = "fra_android_male_24_30"
print(agg_df[agg_df["customers_level_based"] == new_user])

Empty DataFrame
Columns: [customers_level_based, PRICE, SEGMENT]
Index: []
